In [11]:
import polars as pl
import altair as alt

In [12]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [25]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).with_columns(
    pl.col("text").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_text")
).with_columns(
    pl.col("mluvci").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_mluvci")
).drop_nulls(
    subset=['datum']
).collect()

In [26]:
df.sample(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Poslanec Ladislav Šincl""","""2010ps_stenprot_032schuz_s032090.htm""","""/sqw/detail.sqw?id=5507""","""Vážený pane premiére, z vašich členů vlády se dnes omluvili pan ministr Petr Bendl z údajných pracovních důvodů, pan ministr Jiří Besser z údajných pr…",3,"""ps""",2010,32,90,2011-12-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""32. schůze""",true,true,true,"""5507""",256,[],[]
"""Poslankyně Kateřina Valachová""","""2017ps_stenprot_020schuz_s020162.htm""","""/sqw/detail.sqw?id=6337""","""Nyní budeme hlasovat o návrzích D19 a D23 společně jedním hlasováním. Jedná se o pozměňovací návrh Kateřiny Valachové týkající se změn v oblasti kompe…",20,"""ps""",2017,20,162,2018-10-26,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""20. schůze""",true,true,true,"""6337""",30,[],[]
"""Místopředseda PSP František Brožík""","""1998ps_stenprot_047schuz_s047336.htm""","""/sqw/p.sqw?P=432""","""Děkuji vám, pane zpravodaji. Kdo se dále hlásí do rozpravy? Pan poslanec Miroslav Kapoun.""",19,"""ps""",1998,47,336,2002-03-26,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""47. schůze""",true,true,true,null,13,[],[]
"""Poslankyně Zuzana Ožanová""","""2021ps_stenprot_041schuz_s041138.htm""","""/sqw/detail.sqw?id=6474""","""A5 až A10 a A12, pozměňovací návrh hospodářského výboru, původně poslance Stanislava Blahy. Změny se týkají výhradně historických vozidel. Rozvolňuje …",8,"""ps""",2021,41,138,2022-10-14,"""PČR, PS 2021-...""","""41. schůze""",true,true,true,"""6474""",47,[],[]
"""Místopředseda PSP Vojtěch Pikal""","""2017ps_stenprot_013schuz_s013170.htm""","""/sqw/detail.sqw?id=6531""","""Já si znamenám návrh na zamítnutí, budu ho považovat za zdůvodněný, a ptám se tedy, jestli se ještě někdo další hlásí do rozpravy. Nevypadá to, tedy r…",9,"""ps""",2017,13,170,2018-05-29,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""13. schůze""",true,true,true,"""6531""",145,[],[]


In [27]:
len(df)

585180

In [28]:
def zavorka(retezec, doslova=True, frejm=df):
    return frejm.explode("poznamky_text").explode("poznamky_mluvci").filter(
            pl.col('poznamky_text').str.contains(retezec, literal=doslova) 
        |
            pl.col('poznamky_mluvci').str.contains(retezec, literal=doslova) 
    ).unique(
        subset=['datum','cast','poradi']
).sort(by='datum').group_by_dynamic(
    index_column='datum',every='1y').agg(pl.col('text').unique().len()
).join(
    frejm.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()), 
    on='datum', how='left'
).with_columns(
    (pl.col('text') / pl.col('text_right')).alias('podil')
).with_columns(
        pl.lit(retezec).alias('co')
)

In [29]:
'; '.join(df.explode("poznamky_text").group_by(
    "poznamky_text"
).len(
).sort(
    by='len',
    descending=True
).head(500).filter(pl.col('poznamky_text').is_not_null()).select(pl.col('poznamky_text')).to_series().to_list())

'potlesk.; ?; ano.; souhlas.; nesouhlasné.; nesouhlas.; potlesk zprava.; hluk v sále.; ministr: nesouhlas.; slibuji.; nesrozumitelné; předsedající: čas.; smích v sále.; ministr: souhlas.; potlesk poslanců čssd.; potlesk poslanců ods.; potlesk zleva.; upozornění na čas; potlesk poslanců ksčm.; potlesk v části sálu.; souhlasné.; potlesk části poslanců.; předsedající upozorňuje na čas.; 14.40 hodin; 14.50 hodin; ne.; 15.20 hodin; 15.10 hodin; 15.00 hodin; 15.40 hodin; upozornění na čas.; 15.30 hodin; 15.50 hodin; velký hluk v sále.; ministr nedoporučuje.; v sále je velký hluk.; 16.10 hodin; smích.; potlesk z lavic ano.; 16.20 hodin; potlesk poslanců ano.; neutrální.; 17.10 hodin; 16.00 hodin; 17.20 hodin; pobavení v sále.; 16.40 hodin; 17.30 hodin; 16.30 hodin; 10.20 hodin; 10.10 hodin; 11.00 hodin; 10.50 hodin; 17.50 hodin; 16.50 hodin; 11.40 hodin; 17.40 hodin; 11.10 hodin; 14.30 hodin; 11.30 hodin; 11.20 hodin; 10.40 hodin; 10.30 hodin; jednání zahájeno v 9.00 hodin.; 11.50 hodin; 18.0

In [30]:
alt.Chart(
    zavorka("potlesk").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [31]:
alt.Chart(
    zavorka("smích").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [32]:
alt.Chart(
    zavorka("výkřiky").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [33]:
df.filter(pl.col("text").str.contains("[Bb]učení"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Ministr zdravotnictví ČR David Rath""","""2002ps_stenprot_055schuz_s055136.htm""","""http://wtd.vlada.cz/scripts/detail.php?id=9214""","""Děkuji. Je to myslím docela zase signifikantní a typické, když názor vyslovený těmi, kterých se to bytostně týká, tedy lékaři zaměstnanci a zdravotním…",15,"""ps""",2002,55,136,2006-04-21,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""55. schůze""",true,true,true,null,201,[],[]
"""Poslanec David Rath""","""2006ps_stenprot_027schuz_s027049.htm""","""/sqw/detail.sqw?id=5406""","""Dámy a pánové, pane předsedající, já bych prosil, abyste poučil o slušném chování poslance ODS, aby zde nebučeli. Nejsou, jak jsou zvyklí, v chlívě. Z…",3,"""ps""",2006,27,49,2008-01-30,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""27. schůze""",true,true,true,"""5406""",426,[],[]
"""Místopředseda PSP Vojtěch Pikal""","""2017ps_stenprot_040schuz_s040102.htm""","""/sqw/detail.sqw?id=6531""","""Děkuji za slovo. Protože nemáme tu britskou tradici, tak tady neprobíhalo během předchozích projevů bučení a podobně, místo toho všeobecný nezájem pří…",7,"""ps""",2017,40,102,2020-01-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true,true,"""6531""",272,"[""potlesk poslanců pirátů.""]",[]
"""Předsedkyně PSP Markéta Pekarová Adamová""","""2021ps_stenprot_005schuz_s005019.htm""","""/sqw/detail.sqw?id=6147""","""Vzhledem k tomu, že program schůze nebyl schválen a nikdo další se s přednostním právem nehlásí, tak já tuto 5. schůzi Poslanecké sněmovny končím. (Po…",2,"""ps""",2021,5,19,2021-12-15,"""PČR, PS 2021-...""","""5. schůze""",true,true,true,"""6147""",77,"[""poslanci tlučou do lavic, ozývá se bučení a volání."", ""schůze skončila ve 12.14 hodin.""]",[]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6418""",119,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]
"""Poslanec Jakub Michálek""","""2021ps_stenprot_055schuz_s055007.htm""","""/sqw/detail.sqw?id=6477""","""Děkuji za slovo. Vážená paní předsedkyně, vážené kolegyně, vážení kolegové, pojďme prosím nechat v rozpravě vystoupit i ministry, například tady pana …",9,"""ps""",2021,55,7,2023-02-28,"""PČR, PS 2021-...""","""55. schůze""",true,true,true,"""6477""",148,"[""nesouhlas a bučení poslanců ano.""]",[]


In [34]:
alt.Chart(
    zavorka("bučen").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [35]:
df.filter(pl.col("text").str.contains("(Buče",literal=True))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6418""",119,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]


In [36]:
alt.Chart(
    zavorka("posmě[cš]",doslova=False).to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [37]:
df.filter(
    pl.col("datum").dt.year().is_between(1940,1970)
).filter(
    pl.col("text").str.contains("(Smích",literal=True)
)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]


In [38]:
alt.Chart(
    pl.concat([zavorka("smích"), zavorka("potlesk"), zavorka("křik")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [87]:
df.explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains()
)

SyntaxError: incomplete input (3606458955.py, line 1)

In [39]:
alt.Chart(
    pl.concat([zavorka("potlesk zleva"), zavorka("potlesk zprava")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [40]:
def obrat(cislo):
    if cislo >= 1:
        return cislo
    if cislo < 1:
        return -(1/cislo)

In [41]:
obrat(0.1)

-10.0

In [42]:
pravolevo = zavorka(
    "(potlesk zleva|potlesk na levici|potlesk z levé|potlesk vlevo)",doslova=False
).with_columns(
    pl.col("podil").alias("zleva")
).select(
    pl.col(['datum','zleva'])
).join(
    zavorka(
        "(potlesk zprava|potlesk na pravici|potlesk z pravé|potlesk vpravo)", doslova=False
    ).with_columns(
        pl.col("podil").alias("zprava")
    ).select(
        pl.col(['datum','zprava'])
    ), on="datum", how="full"
).with_columns(
    (pl.col('zprava') / pl.col('zleva')).map_elements(obrat, return_dtype=pl.Float32).alias('pravo-levo')
).filter(
    pl.col('datum').dt.year() >= 1993
)
pravolevo

datum,zleva,datum_right,zprava,pravo-levo
date,f64,date,f64,f32
1995-01-01,0.000325,1995-01-01,0.000108,-3.0
1996-01-01,0.000321,1996-01-01,0.000642,2.0
1997-01-01,0.001039,1997-01-01,0.000244,-4.25
1998-01-01,0.001107,1998-01-01,0.000511,-2.166667
1999-01-01,0.001766,1999-01-01,0.00211,1.194444
2000-01-01,0.000927,2000-01-01,0.001054,1.136364
2001-01-01,0.000296,2001-01-01,0.000741,2.5
2002-01-01,0.000371,2002-01-01,0.001237,3.333333
2003-01-01,0.000464,2003-01-01,0.003758,8.1


## Po 9 letech se víc tleská zleva!

In [44]:
alt.Chart(
    pravolevo.to_pandas(),
    title=["„Potlesk zprava“ (> 0) a „potlesk zleva“ (< 0)","ve stenozáznamech PS PSP ČR"]
).mark_bar().encode(alt.Y('datum:T'),alt.X('pravo-levo:Q',axis=alt.Axis(title="<– víc se tleská zleva | víc se tleská zprava ->")))

alt.Chart(...)

In [45]:
alt.Chart(
    pl.concat([zavorka("pobavení"), zavorka("veselost"), zavorka("veselí")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [46]:
alt.Chart(
    pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_20676\4207413198.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
C:\Users\micha\AppData\Local\Temp\ipykernel_20676\4207413198.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [47]:
alt.Chart(
    pl.concat([zavorka("(Smích", doslova=True), zavorka("(Veselí", doslova=True), zavorka("(Pobavení", doslova=True)]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [48]:
alt.Chart(
    df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('text:Q', axis=alt.Axis(title=None)),
)

alt.Chart(...)

In [49]:
df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len())

datum,text
date,u32
1993-01-01,9337
1994-01-01,7730
1995-01-01,9219
1996-01-01,9349
1997-01-01,16367
1998-01-01,11744
1999-01-01,20383
2000-01-01,23721
2001-01-01,20242
